In [6]:
#data source
#https://search.earthdata.nasa.gov/search/granules?p=C1940475563-POCLOUD&pg[0][v]=f&pg[0][gsk]=start_date&g=G1996941394-POCLOUD&tl=1666010677.42!3!!&lat=78.609375&long=0.140625&zoom=1


https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MODIS_T-JPL-L2P-v2019.0/20000224003506-JPL-L2P_GHRSST-SSTskin-MODIS_T-D-v02.0-fv01.0.nc
#https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MODIS_T-JPL-L2P-v2019.0/20000224000006-JPL-L2P_GHRSST-SSTskin-MODIS_T-N-v02.0-fv01.0.nc


#https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html


SyntaxError: invalid syntax (<ipython-input-6-06a2b411352e>, line 5)

In [5]:

from ipynb.fs.full.utils import *
from glob import glob
import os
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re

import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt



In [7]:

fn = '/home/fun/sst/sst.day.mean.2020.v2.nc'
ds = nc.Dataset(fn)


print(ds)
#print(ds.__dict__)
# for dim in ds.dimensions.values():
#     print(dim)
    
# for var in ds.variables.values():
#     print(var)
    
    
#print(ds['sst'])



sst.shape


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.5
    title: NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surface Temperature (OISST) Analysis, Version 2.1
    institution: NOAA/National Centers for Environmental Information
    source: NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/
    References: https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html
    dataset_title: NOAA Daily Optimum Interpolation Sea Surface Temperature
    version: Version 2.1
    comment: Reynolds, et al.(2007) Daily High-Resolution-Blended Analyses for Sea Surface Temperature (available at https://doi.org/10.1175/2007JCLI1824.1). Banzon, et al.(2016) A long-term record of blended satellite and in situ sea-surface temperature for climate monitoring, modeling and environmental studies (available at https://doi.org/10.5194/essd-8-165-2016). Huang et al. (2020) Impr

NameError: name 'sst' is not defined

In [8]:
plt.imshow(sst[250])
plt.gca().invert_yaxis()

NameError: name 'sst' is not defined

In [37]:
def date_to_nth_day(date, fmt='%Y%m%d'):
    date = datetime.datetime.strptime(date, fmt)
    new_year_day = datetime.datetime(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1


def getTgtArea(rgb, lon, lat, radias):
    '''
    lat (-90, 90)
    lon (0, E(0, 180), W(0, 180))
    '''
    n = 720
    m = 1440
    
    interval = 180.0 / n
    
    lat_index = int((90 + lat) / interval)

    long_index = int((lon + 360) / interval) # only works for West
    
    n_raidus = int(radias / interval)
    
    
    return rgb[lat_index - n_raidus:lat_index + n_raidus, long_index - n_raidus:long_index + n_raidus]
def generate_noaa_filename(datestr):
    '''
    compose url using date  'YYYYMMDD'
    '''
    fname = '/home/fun/sst/sst.day.mean.' + datestr[0:4] + '.v2.nc'
    return fname
    #datestr[4:6] + '/' + datestr[6:8]
    
def post_processing(folder, sst, clip):
    idx = date_to_nth_day(str(clip['date']))  
    area = getTgtArea(sst[idx], -1 * float(clip['lg'][:-1]), float(clip['lt'][:-1]), 12.5)
    print(type(area))
    
    final = np.array([area])
    pathname = folder + '/noaa_sst_' + str(clip['date']) + '.npy'
    np.save(pathname, final)
    
#   call it noaa_sst_
    
   
def download_noaa(folder):
    json_profile = folder + '/profile.json'
    json_file = open(json_profile)
    d = json.load(json_file)
        
    for it in d['path']:
        print(it)
        fname = generate_noaa_filename(str(it['date']))
        
        ds = nc.Dataset(fname)
        sst = ds['sst']
        post_processing(folder, sst, it)
        


In [38]:

hurricanes_folders = glob("/home/fun/data/AL*2012", recursive = True)


for it in hurricanes_folders:
    sst = download_noaa(it)


{'date': 20120519, 'time': '0000', 'category': 'LO', 'lt': '33.1N', 'lg': '77.0W', 'maxwind': '25', 'maxpres': '1010', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120519, 'time': '0600', 'category': 'TD', 'lt': '32.8N', 'lg': '77.1W', 'maxwind': '30', 'maxpres': '1008', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120519, 'time': '1200', 'category': 'TS', 'lt': '32.5N', 'lg': '77.3W', 'maxwind': '40', 'maxpres': '1005', '34ktne': '30', '34ktse': '0', '34ktsw': '0', '34ktnw': '30', '50ktne': '30', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0',

<class 'numpy.ma.core.MaskedArray'>
{'date': 20120527, 'time': '0600', 'category': 'SS', 'lt': '30.7N', 'lg': '78.0W', 'maxwind': '45', 'maxpres': '998', '34ktne': '80', '34ktse': '50', '34ktsw': '50', '34ktnw': '50', '50ktne': '80', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120527, 'time': '1200', 'category': 'SS', 'lt': '30.3N', 'lg': '78.9W', 'maxwind': '50', 'maxpres': '998', '34ktne': '80', '34ktse': '80', '34ktsw': '80', '34ktnw': '60', '50ktne': '80', '50ktse': '50', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120527, 'time': '1800', 'category': 'TS', 'lt': '30.1N', 'lg': '79.8W', 'maxwind': '55', 'maxpres': '997', '34ktne': '100', '34ktse': '100', '34ktsw': '100', '34ktnw': '60', '50ktne': '80', '50ktse': '50', '50ktsw': '0', '50ktnw':

<class 'numpy.ma.core.MaskedArray'>
{'date': 20120617, 'time': '0600', 'category': 'LO', 'lt': '29.3N', 'lg': '66.9W', 'maxwind': '25', 'maxpres': '1008', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120617, 'time': '1200', 'category': 'LO', 'lt': '30.4N', 'lg': '65.4W', 'maxwind': '30', 'maxpres': '1007', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120617, 'time': '1800', 'category': 'LO', 'lt': '31.8N', 'lg': '64.2W', 'maxwind': '35', 'maxpres': '1004', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '40', '50ktne': '40', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne':

<class 'numpy.ma.core.MaskedArray'>
{'date': 20120624, 'time': '0600', 'category': 'EX', 'lt': '42.9N', 'lg': '38.5W', 'maxwind': '25', 'maxpres': '993', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120624, 'time': '1200', 'category': 'EX', 'lt': '43.7N', 'lg': '38.2W', 'maxwind': '20', 'maxpres': '995', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120623, 'time': '1200', 'category': 'TS', 'lt': '25.5N', 'lg': '87.6W', 'maxwind': '35', 'maxpres': '1002', '34ktne': '150', '34ktse': '150', '34ktsw': '150', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne

<class 'numpy.ma.core.MaskedArray'>
{'date': 20120804, 'time': '0000', 'category': 'TS', 'lt': '13.8N', 'lg': '64.8W', 'maxwind': '40', 'maxpres': '1004', '34ktne': '100', '34ktse': '80', '34ktsw': '80', '34ktnw': '0', '50ktne': '50', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120804, 'time': '0600', 'category': 'TS', 'lt': '13.9N', 'lg': '66.4W', 'maxwind': '45', 'maxpres': '1005', '34ktne': '100', '34ktse': '80', '34ktsw': '80', '34ktnw': '0', '50ktne': '50', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20120804, 'time': '1200', 'category': 'TS', 'lt': '14.2N', 'lg': '67.9W', 'maxwind': '45', 'maxpres': '1005', '34ktne': '100', '34ktse': '80', '34ktsw': '80', '34ktnw': '0', '50ktne': '50', '50ktse': '0', '50ktsw': '0', '50ktnw': '

...

<class 'numpy.ma.core.MaskedArray'>
{'date': 20121015, 'time': '0000', 'category': 'TS', 'lt': '21.3N', 'lg': '64.7W', 'maxwind': '60', 'maxpres': '989', '34ktne': '150', '34ktse': '150', '34ktsw': '150', '34ktnw': '0', '50ktne': '100', '50ktse': '70', '50ktsw': '0', '50ktnw': '0', '64ktne': '30', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121015, 'time': '0600', 'category': 'HU', 'lt': '22.0N', 'lg': '65.2W', 'maxwind': '65', 'maxpres': '986', '34ktne': '150', '34ktse': '150', '34ktsw': '150', '34ktnw': '50', '50ktne': '100', '50ktse': '70', '50ktsw': '40', '50ktnw': '15', '64ktne': '40', '64ktse': '40', '64ktsw': '20', '64ktnw': '0', 'maxwindradius': '30'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121015, 'time': '1200', 'category': 'HU', 'lt': '22.6N', 'lg': '65.6W', 'maxwind': '70', 'maxpres': '983', '34ktne': '180', '34ktse': '150', '34ktsw': '150', '34ktnw': '80', '50ktne': '120', '50ktse': '90', '50ktsw'

{'date': 20121021, 'time': '0600', 'category': 'EX', 'lt': '49.2N', 'lg': '31.1W', 'maxwind': '40', 'maxpres': '978', '34ktne': '0', '34ktse': '240', '34ktsw': '240', '34ktnw': '480', '50ktne': '480', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121021, 'time': '1200', 'category': 'EX', 'lt': '48.2N', 'lg': '30.6W', 'maxwind': '40', 'maxpres': '979', '34ktne': '0', '34ktse': '240', '34ktsw': '240', '34ktnw': '480', '50ktne': '480', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121021, 'time': '1800', 'category': 'EX', 'lt': '47.0N', 'lg': '30.8W', 'maxwind': '40', 'maxpres': '982', '34ktne': '0', '34ktse': '240', '34ktsw': '240', '34ktnw': '480', '50ktne': '480', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '

<class 'numpy.ma.core.MaskedArray'>
{'date': 20121023, 'time': '0000', 'category': 'TS', 'lt': '12.6N', 'lg': '78.4W', 'maxwind': '40', 'maxpres': '998', '34ktne': '50', '34ktse': '60', '34ktsw': '60', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121023, 'time': '0600', 'category': 'TS', 'lt': '12.9N', 'lg': '78.1W', 'maxwind': '40', 'maxpres': '998', '34ktne': '70', '34ktse': '80', '34ktsw': '80', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121023, 'time': '1200', 'category': 'TS', 'lt': '13.4N', 'lg': '77.9W', 'maxwind': '40', 'maxpres': '995', '34ktne': '100', '34ktse': '100', '34ktsw': '100', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '6

<class 'numpy.ma.core.MaskedArray'>
{'date': 20121029, 'time': '0600', 'category': 'HU', 'lt': '35.3N', 'lg': '70.5W', 'maxwind': '80', 'maxpres': '947', '34ktne': '420', '34ktse': '360', '34ktsw': '360', '34ktnw': '450', '50ktne': '270', '50ktse': '150', '50ktsw': '150', '50ktnw': '200', '64ktne': '180', '64ktse': '0', '64ktsw': '0', '64ktnw': '150', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121029, 'time': '1200', 'category': 'HU', 'lt': '36.9N', 'lg': '71.0W', 'maxwind': '85', 'maxpres': '945', '34ktne': '420', '34ktse': '420', '34ktsw': '420', '34ktnw': '400', '50ktne': '270', '50ktse': '200', '50ktsw': '150', '50ktnw': '200', '64ktne': '180', '64ktse': '0', '64ktsw': '70', '64ktnw': '120', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121029, 'time': '1800', 'category': 'HU', 'lt': '38.3N', 'lg': '73.2W', 'maxwind': '80', 'maxpres': '940', '34ktne': '420', '34ktse': '420', '34ktsw': '420', '34ktnw': '400', '50ktne': '420', '50ktse'

<class 'numpy.ma.core.MaskedArray'>
{'date': 20121025, 'time': '1200', 'category': 'TS', 'lt': '30.6N', 'lg': '37.6W', 'maxwind': '35', 'maxpres': '1000', '34ktne': '80', '34ktse': '70', '34ktsw': '70', '34ktnw': '40', '50ktne': '30', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121025, 'time': '1800', 'category': 'EX', 'lt': '31.0N', 'lg': '35.1W', 'maxwind': '35', 'maxpres': '1001', '34ktne': '60', '34ktse': '70', '34ktsw': '70', '34ktnw': '20', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
<class 'numpy.ma.core.MaskedArray'>
{'date': 20121026, 'time': '0000', 'category': 'EX', 'lt': '31.3N', 'lg': '32.6W', 'maxwind': '35', 'maxpres': '1001', '34ktne': '60', '34ktse': '70', '34ktsw': '70', '34ktnw': '20', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0'